### 1A

In [31]:
library(mlogit)
options(repr.plot.width=7, repr.plot.height=4) # resize IRkernel plot size
heating <- read.csv("csv/Heating.csv")
str(heating)
head(heating)

# ?mlogit.data
# this preprocesses the heating dataset to be ready to be inputted into the mlogit model
data1A <- mlogit.data(heating,
                 shape = "wide",
                 choice = "depvar",
                 alt.levels = c("gc", "gr", "ec", "er", "hp"),
                 sep = ".",
                 id.var = "idcase",
                 varying = c(3:12))
head(data1A)

# no closed-form solution for the mlogit model, just like logistic regression
model1A <- mlogit(depvar~ic+oc-1, data=data1A)
summary(model1A)

'data.frame':	900 obs. of  16 variables:
 $ idcase: int  1 2 3 4 5 6 7 8 9 10 ...
 $ depvar: Factor w/ 5 levels "ec","er","gc",..: 3 3 3 2 2 3 3 3 3 3 ...
 $ ic.gc : num  866 728 599 835 756 ...
 $ ic.gr : num  963 759 783 793 846 ...
 $ ic.ec : num  860 797 720 761 859 ...
 $ ic.er : num  996 895 900 831 986 ...
 $ ic.hp : num  1136 969 1048 1049 883 ...
 $ oc.gc : num  200 169 166 181 175 ...
 $ oc.gr : num  152 169 138 147 139 ...
 $ oc.ec : num  553 520 439 483 404 ...
 $ oc.er : num  506 486 405 425 390 ...
 $ oc.hp : num  238 199 171 223 178 ...
 $ income: int  7 5 4 2 2 6 4 6 5 7 ...
 $ agehed: int  25 60 65 50 25 65 35 20 60 20 ...
 $ rooms : int  6 5 2 4 6 7 2 7 6 2 ...
 $ region: Factor w/ 4 levels "mountn","ncostl",..: 2 3 2 3 4 3 3 4 3 3 ...


idcase,depvar,ic.gc,ic.gr,ic.ec,ic.er,ic.hp,oc.gc,oc.gr,oc.ec,oc.er,oc.hp,income,agehed,rooms,region
1,gc,866.00,962.64,859.90,995.76,1135.50,199.69,151.72,553.34,505.60,237.88,7,25,6,ncostl
2,gc,727.93,758.89,796.82,894.69,968.90,168.66,168.66,520.24,486.49,199.19,5,60,5,scostl
3,gc,599.48,783.05,719.86,900.11,1048.30,165.58,137.80,439.06,404.74,171.47,4,65,2,ncostl
4,er,835.17,793.06,761.25,831.04,1048.70,180.88,147.14,483.00,425.22,222.95,2,50,4,scostl
5,er,755.59,846.29,858.86,985.64,883.05,174.91,138.90,404.41,389.52,178.49,2,25,6,valley
6,gc,666.11,841.71,693.74,862.56,859.18,135.67,140.97,398.22,371.04,209.27,6,65,7,scostl


,idcase,depvar,income,agehed,rooms,region,alt,ic,oc,chid
1.gc,1,FALSE,7,25,6,ncostl,ec,859.90,553.34,1
1.gr,1,FALSE,7,25,6,ncostl,er,995.76,505.60,1
1.ec,1,TRUE,7,25,6,ncostl,gc,866.00,199.69,1
1.er,1,FALSE,7,25,6,ncostl,gr,962.64,151.72,1
1.hp,1,FALSE,7,25,6,ncostl,hp,1135.50,237.88,1
2.gc,2,FALSE,5,60,5,scostl,ec,796.82,520.24,2



Call:
mlogit(formula = depvar ~ ic + oc - 1, data = data1A, method = "nr", 
    print.level = 0)

Frequencies of alternatives:
      gc       gr       ec       er       hp 
0.071111 0.093333 0.636667 0.143333 0.055556 

nr method
4 iterations, 0h:0m:0s 
g'(-H)^-1g = 1.56E-07 
gradient close to zero 

Coefficients :
      Estimate  Std. Error t-value  Pr(>|t|)    
ic -0.00623187  0.00035277 -17.665 < 2.2e-16 ***
oc -0.00458008  0.00032216 -14.217 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likelihood: -1095.2

Both the coefficients of `ic` (installation cost) and `oc` (operating cost) are negative, which makes sense since the more expensive the alternative is, the less likely one will want to purchase it all else considered equal.

Given the extremely small p-value, we can be confident in saying that the coefficient values are in fact non-zero, albeit with a tiny magnitude.

In [32]:
### SUBPART III
table(heating$depvar)/NROW(heating) # actual share
predict1A <- predict(model1A, data1A)
apply(predict1A, 2, mean) # predicted share


        ec         er         gc         gr         hp 
0.07111111 0.09333333 0.63666667 0.14333333 0.05555556 

gc         gr         ec         er         hp 
0.10413057 0.05141477 0.51695653 0.24030898 0.08718915

We can see that the predicted share do not bear any resemblance the actual share at all.

In [3]:
### SUBPART IV
wtp1A <- model1A$coef["oc"]/model1A$coef["ic"]
wtp1A

oc 
0.7349453

For every dollar decrease increase in upfront installation cost, the owner expects roughly $0.73 decrease in yearly operational costs for the alternative to be equivalent.

### 1B

In [8]:
# new lifecycle cost
r = 0.12 # discount rate
data1B <- data1A
data1B$lcc <- data1B$ic + data1B$oc/r
model1B <- mlogit(depvar~lcc-1 ,data1B)

table(heating$depvar)/NROW(heating) # actual share
predict1B <- predict(model1B, data1B)
apply(predict1B, 2, mean) # predicted share

# get log-likelihood
logLik(model1A)
logLik(model1B)


        ec         er         gc         gr         hp 
0.07111111 0.09333333 0.63666667 0.14333333 0.05555556 

gc         gr         ec         er         hp 
0.05503174 0.06441002 0.33538584 0.33599396 0.20917844

'log Lik.' -1095.237 (df=2)

'log Lik.' -1248.702 (df=1)

### 1C

Adding alternative-specific constants merely means adding the relevant intercepts/constants to the utility equations for each alternative, with reference to a baseline alternative that has an alternative-specific constant value of 0. E.g. (where alternative m is the reference level):

- **Alternative k**: $U_{k} = \beta'x_k + C_k + \epsilon_k$, where $C_k=$ alternative-specific constant for k
- **Alternative l**: $U_{l} = \beta'x_l + C_l + \epsilon_l$, where $C_l=$ alternative-specific constant for l
- **Alternative m**: $U_{m} = \beta'x_m + \epsilon_l$

**Q:** what is the operational difference between `+1` and adding `reflevel="hp"` for the model in `mlogit()`?

**A:** it seems by adding `+1` the `mlogit()` function is automatically treating it as adding alternative-specific constants with a randomly-chosen reference alternative. Note there is no difference in results even if normalisation is not carried out; it is merely convenient for us.

In [39]:
### SUBPART I
model1C <- mlogit(depvar~ic+oc, data=data1B, reflevel="hp")
summary(model1C)
predict1C <- predict(model1C, data1B)
# comparing shares
table(heating$depvar)/NROW(heating) # actual share
apply(predict1C, 2, mean) # predicted share

# here, in fact the predicted shares are exactly identical to the actual share


Call:
mlogit(formula = depvar ~ ic + oc, data = data1B, reflevel = "hp", 
    method = "nr", print.level = 0)

Frequencies of alternatives:
      hp       gc       gr       ec       er 
0.055556 0.071111 0.093333 0.636667 0.143333 

nr method
6 iterations, 0h:0m:0s 
g'(-H)^-1g = 9.58E-06 
successive function values within tolerance limits 

Coefficients :
                  Estimate  Std. Error t-value  Pr(>|t|)    
gc:(intercept)  1.65884594  0.44841936  3.6993 0.0002162 ***
gr:(intercept)  1.85343697  0.36195509  5.1206 3.045e-07 ***
ec:(intercept)  1.71097930  0.22674214  7.5459 4.485e-14 ***
er:(intercept)  0.30826328  0.20659222  1.4921 0.1356640    
ic             -0.00153315  0.00062086 -2.4694 0.0135333 *  
oc             -0.00699637  0.00155408 -4.5019 6.734e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likelihood: -1008.2
McFadden R^2:  0.013691 
Likelihood ratio test : chisq = 27.99 (p.value = 8.3572e-07)


        ec         er         gc         gr         hp 
0.07111111 0.09333333 0.63666667 0.14333333 0.05555556 

hp         gc         gr         ec         er 
0.05555556 0.07111111 0.09333333 0.63666666 0.14333334

In [35]:
### SUBPART II
model1C$coef["oc"]/model1C$coef["ic"]

oc 
4.563385

For every dollar increase in upfront installation cost, the owner expects roughly $4.56 decrease in yearly operational costs for the alternative to be equivalent. This is quite unreasonable and economically unsound since suppliers will clearly not make profits if this were to be true.

In [40]:
### SUBPART III
# note that in model1C, gc-value = 1.659
# we want to reduce gr-value to 0.
# therefore, gr-value in new model = 1.659-1.853
1.659-1.853

[1] -0.194

Note that appending constants to ALL alternatives on top of already-added alternative constants won't change probabilities

### 1D

Compared to Model1C, Model1D has a lower log-likelihood and is hence a worse fit since we want to maximise log-likelihood. Also, the augmented income/installation-cost variable is no longer significant.

In [46]:
### SUBPART 1
data1A$iic <- data1A$ic/data1A$income
model1D1 <- mlogit(depvar~iic+oc, data1A)
summary(model1D1)
summary(model1C)


Call:
mlogit(formula = depvar ~ iic + oc, data = data1A, method = "nr", 
    print.level = 0)

Frequencies of alternatives:
      gc       gr       ec       er       hp 
0.071111 0.093333 0.636667 0.143333 0.055556 

nr method
6 iterations, 0h:0m:0s 
g'(-H)^-1g = 1.03E-05 
successive function values within tolerance limits 

Coefficients :
                 Estimate Std. Error t-value  Pr(>|t|)    
gr:(intercept)  0.0639934  0.1944893  0.3290  0.742131    
ec:(intercept)  0.0563481  0.4650251  0.1212  0.903555    
er:(intercept) -1.4653063  0.5033845 -2.9109  0.003604 ** 
hp:(intercept) -1.8700773  0.4364248 -4.2850 1.827e-05 ***
iic            -0.0027658  0.0018944 -1.4600  0.144298    
oc             -0.0071066  0.0015518 -4.5797 4.657e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likelihood: -1010.2
McFadden R^2:  0.011765 
Likelihood ratio test : chisq = 24.052 (p.value = 5.9854e-06)


Call:
mlogit(formula = depvar ~ ic + oc, data = data1B, reflevel = "hp", 
    method = "nr", print.level = 0)

Frequencies of alternatives:
      hp       gc       gr       ec       er 
0.055556 0.071111 0.093333 0.636667 0.143333 

nr method
6 iterations, 0h:0m:0s 
g'(-H)^-1g = 9.58E-06 
successive function values within tolerance limits 

Coefficients :
                  Estimate  Std. Error t-value  Pr(>|t|)    
gc:(intercept)  1.65884594  0.44841936  3.6993 0.0002162 ***
gr:(intercept)  1.85343697  0.36195509  5.1206 3.045e-07 ***
ec:(intercept)  1.71097930  0.22674214  7.5459 4.485e-14 ***
er:(intercept)  0.30826328  0.20659222  1.4921 0.1356640    
ic             -0.00153315  0.00062086 -2.4694 0.0135333 *  
oc             -0.00699637  0.00155408 -4.5019 6.734e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likelihood: -1008.2
McFadden R^2:  0.013691 
Likelihood ratio test : chisq = 27.99 (p.value = 8.3572e-07)

In [52]:
### SUBPART II
model1D2 <- mlogit(depvar~oc+ic|income, data1A)
# model1D2 <- mlogit(depvar~oc|income+ic, data1A)
summary(model1D2)

# this estimates a coefficient for each  alternative that is income dependent.
# As income rises, probability of choosing `hp` increases relative to others
# As income rises, probability of choosing `gr` decreases relative to others
# None of these variables are significant however


Call:
mlogit(formula = depvar ~ oc | income + ic, data = data1A, method = "nr", 
    print.level = 0)

Frequencies of alternatives:
      gc       gr       ec       er       hp 
0.071111 0.093333 0.636667 0.143333 0.055556 

nr method
6 iterations, 0h:0m:0s 
g'(-H)^-1g = 5.15E-05 
successive function values within tolerance limits 

Coefficients :
                  Estimate  Std. Error t-value  Pr(>|t|)    
gr:(intercept)  2.23414866  1.00659539  2.2195 0.0264520 *  
ec:(intercept)  0.90508942  0.77064863  1.1745 0.2402142    
er:(intercept) -0.55101204  0.93575437 -0.5888 0.5559669    
hp:(intercept) -0.80603925  1.22000644 -0.6607 0.5088147    
oc             -0.00634691  0.00174082 -3.6459 0.0002664 ***
gr:income      -0.03930273  0.09943773 -0.3952 0.6926586    
ec:income      -0.01035931  0.07860167 -0.1318 0.8951464    
er:income      -0.11847274  0.09118180 -1.2993 0.1938402    
hp:income       0.06051634  0.11318339  0.5347 0.5928745    
gr:ic          -0.00217418  0.00088874 

### 1E

1. We can observe the predicted share of houses with `house pump (hp)` will increase from 5.6% to 6.4%
2. 
3. 

In [78]:
# original model with alternative-specific constants
model1E <- mlogit(depvar~ic+oc, data1A, reflevel="hp")
summary(model1E)
predict1E <- predict(model1E, data1A)
# head(predict1E) # explicit predicted probabilities for each house

# SUBPART I
heating1E <- heating
heating1E$ic.hp <- 0.9*heating1E$ic.hp
data1E1 <- mlogit.data(heating1E,
                    shape = "wide",
                    choice = "depvar",
                    alt.levels = c("gc", "gr", "ec", "er", "hp"),
                    sep = ".",
                    id.var = "idcase",
                    varying = c(3:12))
# now, given new data, what will be the predicted shares under old model
predict1E1 <- predict(model1E, data1E1)
apply(predict1E, 2, mean) # predicted share of `hp` without rebates
apply(predict1E1, 2, mean) # predicted share of `hp` with rebates


Call:
mlogit(formula = depvar ~ ic + oc, data = data1A, reflevel = "hp", 
    method = "nr", print.level = 0)

Frequencies of alternatives:
      hp       gc       gr       ec       er 
0.055556 0.071111 0.093333 0.636667 0.143333 

nr method
6 iterations, 0h:0m:0s 
g'(-H)^-1g = 9.58E-06 
successive function values within tolerance limits 

Coefficients :
                  Estimate  Std. Error t-value  Pr(>|t|)    
gc:(intercept)  1.65884594  0.44841936  3.6993 0.0002162 ***
gr:(intercept)  1.85343697  0.36195509  5.1206 3.045e-07 ***
ec:(intercept)  1.71097930  0.22674214  7.5459 4.485e-14 ***
er:(intercept)  0.30826328  0.20659222  1.4921 0.1356640    
ic             -0.00153315  0.00062086 -2.4694 0.0135333 *  
oc             -0.00699637  0.00155408 -4.5019 6.734e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likelihood: -1008.2
McFadden R^2:  0.013691 
Likelihood ratio test : chisq = 27.99 (p.value = 8.3572e-07)

hp         gc         gr         ec         er 
0.05555556 0.07111111 0.09333333 0.63666666 0.14333334

hp         gc         gr         ec         er 
0.06446230 0.07045486 0.09247026 0.63064443 0.14196814

In [121]:
### SUBPART II
heating1E2 <- heating
heating1E2$ic.eec <- 200 + heating$ic.ec
heating1E2$oc.eec <- 0.75 * heating$oc.ec
# data1E2 <- mlogit.data(heating1E2,
#                     shape = "wide",
#                     choice = "depvar",
#                     alt.levels = c("gc", "gr", "ec", "er", "hp", "eec"),
#                     sep = ".",
#                     id.var = "idcase",
#                     varying = c(3:12, 17, 18))

# model1E$coef[]
exp_gc <- exp(model1E$coef["ic"]*heating1E2$ic.gc+
             model1E$coef["oc"]*heating1E2$oc.gc+
             model1E$coef["gc:(intercept)"])
exp_gr <- exp(model1E$coef["ic"]*heating1E2$ic.gr+
             model1E$coef["oc"]*heating1E2$oc.gr+
             model1E$coef["gr:(intercept)"])
exp_ec <- exp(model1E$coef["ic"]*heating1E2$ic.ec+
             model1E$coef["oc"]*heating1E2$oc.ec+
             model1E$coef["ec:(intercept)"])
exp_er <- exp(model1E$coef["ic"]*heating1E2$ic.er+
             model1E$coef["oc"]*heating1E2$oc.er+
             model1E$coef["er:(intercept)"])
exp_hp <- exp(model1E$coef["ic"]*heating1E2$ic.hp+
             model1E$coef["oc"]*heating1E2$oc.hp)
exp_eec <- exp(model1E$coef["ic"]*heating1E2$ic.eec+
             model1E$coef["oc"]*heating1E2$oc.eec)

denm_old <- exp_gc + exp_gr + exp_ec + exp_er + exp_hp
denm_new <- exp_gc + exp_gr + exp_ec + exp_er + exp_hp + exp_eec

p_gc_old <- exp_gc / denm_old
p_gc_new <- exp_gc / denm_new

p_gr_old <- exp_gr / denm_old
p_gr_new <- exp_gr / denm_new

p_ec_old <- exp_ec / denm_old
p_ec_new <- exp_ec / denm_new

p_er_old <- exp_er / denm_old
p_er_new <- exp_er / denm_new

p_hp_old <- exp_hp / denm_old
p_hp_new <- exp_hp / denm_new

p_eec_new <- exp_eec / denm_new


# old market share
apply(predict1C, 2, mean)
mean(p_hp_old)
mean(p_gc_old)
mean(p_gr_old)
mean(p_ec_old)
mean(p_er_old)

# new market share
print("--")
mean(p_hp_new)
mean(p_gc_new)
mean(p_gr_new)
mean(p_ec_new)
mean(p_er_new)
mean(p_eec_new)
# ?apply


# predict1E2 <- predict(model1E, data1E2)
# apply(predict1E, 2, mean) # predicted share of original `ec` system
# apply(predict1E2, 2, mean) # predicted share of more efficient `ec` system

hp         gc         gr         ec         er 
0.05555556 0.07111111 0.09333333 0.63666666 0.14333334

[1] 0.03909112

[1] 0.4256127

[1] 0.4680874

[1] 0.05304737

...

[1] 0.4191046

[1] 0.4609488

[1] 0.05214355

[1] 0.01393507

[1] 0.01538467